In [2]:
from runner import BENCHMARK_APPLICATIONS
from process_results import *
import os

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [12]:
for name, cls in BENCHMARK_APPLICATIONS.items():
    app = cls()
    static_result_path = app.get_latest_result("static")
    exceptions = read_exceptions(os.path.join(static_result_path, "exception.json"))
    print(name, len(exceptions))


jena_324 6
fineract_1211 834
mapreduce_6654 58
nifi_8249 79
hadoop_17812 33
wicket_6249 10
hdfs_4128 118
hive_13410 2794
